In [2]:

import os
import random
import time
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def greedy_set_cover(universe, sets):
    U_rem = set(universe)
    chosen = []
    while U_rem:
        best = None
        best_cover = set()
        for i, s in enumerate(sets):
            cover = s & U_rem
            if len(cover) > len(best_cover):
                best_cover = cover
                best = i
        if best is None:
            break
        chosen.append(best)
        U_rem -= best_cover
    return chosen

def weighted_greedy_set_cover(universe, sets, costs):
    U_rem = set(universe)
    chosen = []
    while U_rem:
        best = None
        best_value = float('inf')  # cost / newly covered elements
        best_cover = set()
        for i, s in enumerate(sets):
            cover = s & U_rem
            if not cover:
                continue
            value = costs[i] / len(cover)
            if value < best_value:
                best_value = value
                best = i
                best_cover = cover
        if best is None:
            break
        chosen.append(best)
        U_rem -= best_cover
    return chosen

def exact_set_cover_bitmask(n, sets):
    
    m = len(sets)
    bitmasks = []
    for s in sets:
        mask = 0
        for e in s:
            mask |= (1 << e)
        bitmasks.append(mask)

    INF = 10**9
    DP = [INF] * (1 << n)
    DP[0] = 0
    prev = [-1] * (1 << n)
    prev_set = [-1] * (1 << n)

    for mask in range(1 << n):
        if DP[mask] == INF:
            continue
        for i, b in enumerate(bitmasks):
            new = mask | b
            if DP[new] > DP[mask] + 1:
                DP[new] = DP[mask] + 1
                prev[new] = mask
                prev_set[new] = i

    full = (1 << n) - 1
    if DP[full] >= INF:
        return None, None
    # reconstruct chosen sets
    chosen = []
    cur = full
    while cur != 0:
        chosen.append(prev_set[cur])
        cur = prev[cur]
    chosen.reverse()
    return DP[full], chosen

def generate_instance(n, m, density=0.3, weighted=False, cost_range=(1,10)):
    universe = list(range(n))
    sets = []
    for _ in range(m):
        s = set()
        for e in universe:
            if random.random() < density:
                s.add(e)
        if not s:
            s.add(random.choice(universe))
        sets.append(s)
    costs = None
    if weighted:
        costs = [random.randint(cost_range[0], cost_range[1]) for _ in range(m)]
    else:
        costs = [1] * m
    return universe, sets, costs

def run_experiments(out_dir="setcover_experiments", seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.makedirs(out_dir, exist_ok=True)

    results = []
    instances_saved = []

    n_list = [10, 12, 14, 16]  
    density_list = [0.15, 0.25, 0.35, 0.5]
    reps = 10

    for n in n_list:
        for density in density_list:
            for rep in range(reps):
                m = max(n, int(n * (1.5 + random.random() * 1.5))) 
                universe, sets, costs = generate_instance(n, m, density=density, weighted=True, cost_range=(1,10))
                inst_id = f"n{n}_d{int(density*100)}_r{rep}"
                inst_path = os.path.join(out_dir, f"instance_{inst_id}.json")
                with open(inst_path, "w") as f:
                    json.dump({
                        "n": n,
                        "m": m,
                        "density": density,
                        "sets": [sorted(list(s)) for s in sets],
                        "costs": costs
                    }, f)
                instances_saved.append(inst_path)

                t0 = time.perf_counter()
                opt_size, opt_chosen = exact_set_cover_bitmask(n, sets)
                t1 = time.perf_counter()
                exact_time = t1 - t0

                t0 = time.perf_counter()
                g_chosen = greedy_set_cover(universe, sets)
                t1 = time.perf_counter()
                greedy_time = t1 - t0
                greedy_size = len(g_chosen) if g_chosen is not None else None

                t0 = time.perf_counter()
                wg_chosen = weighted_greedy_set_cover(universe, sets, costs)
                t1 = time.perf_counter()
                wg_time = t1 - t0
                wg_size = len(wg_chosen) if wg_chosen is not None else None

                results.append({
                    "instance": inst_id,
                    "n": n,
                    "m": m,
                    "density": density,
                    "opt_size": opt_size,
                    "exact_time": exact_time,
                    "greedy_size": greedy_size,
                    "greedy_time": greedy_time,
                    "wg_size": wg_size,
                    "wg_time": wg_time
                })

    df = pd.DataFrame(results)
    csv_path = os.path.join(out_dir, "experiment_results.csv")
    df.to_csv(csv_path, index=False)


    plt.figure(figsize=(6, 4))
    grouped = df.groupby("n").apply(
        lambda g: (g["greedy_size"] / g["opt_size"]).replace([np.inf, np.nan], np.nan).dropna().values
    )
    x = []
    y_mean = []
    y_median = []
    for n, arr in grouped.items():
        arr = np.array(arr)
        x.append(n)
        y_mean.append(np.mean(arr))
        y_median.append(np.median(arr))
    plt.plot(x, y_mean, marker='o', label='mean ratio')
    plt.plot(x, y_median, marker='s', label='median ratio')
    plt.xlabel("Universe size n")
    plt.ylabel("Greedy / Opt ratio")
    plt.title("Greedy Approximation Ratio vs n")
    plt.legend()
    plt.tight_layout()
    ratio_vs_n = os.path.join(out_dir, "ratio_vs_n.png")
    plt.savefig(ratio_vs_n)
    plt.close()

    plt.figure(figsize=(6, 4))
    rt = df.groupby("n")[["exact_time", "greedy_time", "wg_time"]].mean().reset_index()
    plt.plot(rt["n"], rt["exact_time"], marker='o', label='exact DP time')
    plt.plot(rt["n"], rt["greedy_time"], marker='s', label='greedy time')
    plt.plot(rt["n"], rt["wg_time"], marker='^', label='weighted greedy time')
    plt.xlabel("Universe size n")
    plt.ylabel("Time (seconds)")
    plt.title("Runtime Comparison: Exact vs Greedy")
    plt.legend()
    plt.tight_layout()
    runtime_comparison = os.path.join(out_dir, "runtime_comparison.png")
    plt.savefig(runtime_comparison)
    plt.close()

    plt.figure(figsize=(6, 4))
    sub = df[df["n"] == 14]
    ratios = (sub["greedy_size"] / sub["opt_size"]).replace([np.inf, np.nan], np.nan).dropna().values
    plt.hist(ratios, bins=10, color='skyblue', edgecolor='black')
    plt.xlabel("Greedy / Opt ratio")
    plt.ylabel("Count")
    plt.title("Histogram of Approximation Ratios (n=14)")
    plt.tight_layout()
    ratio_histogram = os.path.join(out_dir, "ratio_histogram.png")
    plt.savefig(ratio_histogram)
    plt.close()

    plt.figure(figsize=(6, 4))
    sub = df[df["n"] == 16]
    densities_sorted = sorted(sub["density"].unique())
    groups = [sub[sub["density"] == d]["greedy_size"] / sub[sub["density"] == d]["opt_size"] for d in densities_sorted]
    groups_clean = [g.replace([np.inf, np.nan], np.nan).dropna().values for g in groups]
    plt.boxplot(groups_clean, labels=[f"{int(d*100)}%" for d in densities_sorted])
    plt.xlabel("Density of sets")
    plt.ylabel("Greedy / Opt ratio")
    plt.title("Density Sweep (n=16)")
    plt.tight_layout()
    density_sweep = os.path.join(out_dir, "density_sweep.png")
    plt.savefig(density_sweep)
    plt.close()

    readme_path = os.path.join(out_dir, "README.txt")
    with open(readme_path, "w") as f:
        f.write("Generated experiment files for Set Cover / Wildlife Sensor Placement.\n")
        f.write("Files:\n")
        f.write("- experiment_results.csv : main CSV with metrics\n")
        f.write("- instance_*.json : saved random instances\n")
        f.write("- ratio_vs_n.png\n")
        f.write("- runtime_comparison.png\n")
        f.write("- ratio_histogram.png\n")
        f.write("- density_sweep.png\n")

    print("Wrote outputs to:", out_dir)
    print("CSV:", csv_path)
    print("Plots:", ratio_vs_n, runtime_comparison, ratio_histogram, density_sweep)


if __name__ == "__main__":
    run_experiments()


Wrote outputs to: setcover_experiments
CSV: setcover_experiments\experiment_results.csv
Plots: setcover_experiments\ratio_vs_n.png setcover_experiments\runtime_comparison.png setcover_experiments\ratio_histogram.png setcover_experiments\density_sweep.png
